# Topic Modeling for youth alcohol consumption - Bertopic

In [1]:
# parameters
nr_of_articles = 10500
startyear = 2000 
nr_topics = 11
min_topic_size = 25

# hyperparameter tuning of Bertopic: https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html

In [2]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


## Data

Searchstring 1 Web of Science: alcohol* AND (adolescen* OR youth OR minor OR underage*) AND (factors OR determinant* OR Cause* OR influence* OR Predictor*)
 
zoeken op Topic  (--> Searches title, abstract and author keywords.)

Dit geeft: n = 23.260 (artikels)

inclusie/exclusiecriteria
Refine: open acces (n=11.605)
Refine: english (n=11.260)
Refine: article + early acces (n = 10,436) --> alle andere bronnen geëxcludeerd op deze wijze

Sort by relevance. 

Searchstring 2 Web of Science: (alcohol*) AND (adolescen* OR youth OR minor OR underage*) AND (factor* OR determinant* OR Cause* OR influence* OR Predictor*)
 
zoeken op Title

languages: english,
open acces, 
Document typers: article





In [3]:
art1 = pd.read_excel('datasets/savedrecs-0001-1000.xls')
art2 = pd.read_excel('datasets/savedrecs-1001-2000.xls')
art3 = pd.read_excel('datasets/savedrecs-2001-3000.xls')
art4 = pd.read_excel('datasets/savedrecs-3001-4000.xls')
art5 = pd.read_excel('datasets/savedrecs-4001-5000.xls')
art6 = pd.read_excel('datasets/savedrecs-5001-5596.xls')    
articles = pd.concat([art1, art2, art3, art4, art5, art6], ignore_index=True, axis=0)

In [4]:
articles = articles.iloc[:nr_of_articles, :]

In [5]:
articles = articles[articles['Publication Year'] >= startyear]

In [6]:
len(articles)

5597

In [7]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5597 entries, 0 to 5596
Data columns (total 72 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            5597 non-null   object 
 1   Authors                     5597 non-null   object 
 2   Book Authors                0 non-null      float64
 3   Book Editors                5 non-null      object 
 4   Book Group Authors          0 non-null      float64
 5   Author Full Names           5597 non-null   object 
 6   Book Author Full Names      0 non-null      float64
 7   Group Authors               104 non-null    object 
 8   Article Title               5597 non-null   object 
 9   Source Title                5597 non-null   object 
 10  Book Series Title           5 non-null      object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    5597 non-null   object 
 13  Document Type               5597 

## Preprocessing

In [8]:
print(len(articles))
articles.drop_duplicates(subset=['Article Title', 'Abstract'], inplace=True)
print(len(articles))

5597
5596


In [9]:
art = articles
art.head(2)

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,Book Series Title,Book Series Subtitle,Language,Document Type,Conference Title,Conference Date,Conference Location,Conference Sponsor,Conference Host,Author Keywords,Keywords Plus,Abstract,Addresses,Affiliations,Reprint Addresses,Email Addresses,Researcher Ids,ORCIDs,Funding Orgs,Funding Name Preferred,Funding Text,Cited References,Cited Reference Count,"Times Cited, WoS Core","Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publisher,Publisher City,Publisher Address,ISSN,eISSN,ISBN,Journal Abbreviation,Journal ISO Abbreviation,Publication Date,Publication Year,Volume,Issue,Part Number,Supplement,Special Issue,Meeting Abstract,Start Page,End Page,Article Number,DOI,DOI Link,Book DOI,Early Access Date,Number of Pages,WoS Categories,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record
0,J,"Shin, YJ; Lee, JK; Lu, Y; Hecht, ML",NaN,NaN,NaN,"Shin, YoungJu; Lee, Jeong-Kyu; Lu, Yu; Hecht, Michael L.",NaN,NaN,Exploring Parental Influence on the Progression of Alcohol Use in Mexican-Heritage Youth: a Latent Transition Analysis,PREVENTION SCIENCE,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,Youth substance use; Latent class analysis; Latent transition analysis; Targeted parent-child communication; Parental monitoring,SUBSTANCE USE; DRUG-USE; CHILD COMMUNICATION; FAMILY-STRUCTURE; SAS PROCEDURE; LATINO YOUTH; SELF-ESTEEM; ADOLESCENTS; RISK; MARIJUANA,"Mexican-heritage youth are members of the fastest growing minority group and are at particular risk for substance use including alcohol consumption. Youth face numerous risk factors including positive descriptions of substance use on media and peer offers that are potentially ameliorated by parental anti-substance use socialization efforts. Guided by primary socialization theory and the theory of planned behavior, the present study posited eight research questions to identify discrete subgroups/patterns of Mexican-heritage youth alcohol use behavior and parental influence on youth outcomes. Longitudinal survey data (n = 1147) from youth in 29 public schools located in Phoenix, Arizona, were collected over 3 years. Latent class and transition analyses identified four discrete subgroups characterized by response patterns of alcohol use behaviors and perceptions in Mexican-heritage youth: (1) non-drinker, (2) potential drinker, (3) experimenter, and (4) regular drinker. Targeted parent-child communication about alcohol and parental monitoring were found to be significant predictors for youth alcohol use. Research implications and future directions are suggested.","[Shin, YoungJu] Indiana Univ Purdue Univ, Dept Commun Studies, 425 Univ Blvd,Cavanaugh Hall 309, Indianapolis, IN 46202 USA; [Lee, Jeong-Kyu] Univ Wollongong, Ctr Hlth Initiat, Wollongong, NSW, Australia; [Lu, Yu; Hecht, Michael L.] Penn State Univ, Dept Commun Arts & Sci, University Pk, PA 16802 USA",Indiana University System; Indiana University-Purdue University Indianapolis; University of Wollongong; Pennsylvania Commonwealth System of Higher Education (PCSHE); Pennsylvania State University; Pennsylvania State University - University Park,"Shin, YJ (corresponding author), Indiana Univ Purdue Univ, Dept Commun Studies, 425 Univ Blvd,Cavanaugh Hall 309, Indianapolis, IN 46202 USA.",younshin@iupui.edu; jklee@uow.edu.au; yxl235@psu.edu; mlh10@psu.edu,"Lu, Yu/AGB-5690-2022; Shin, YoungJu/ABA-1000-2020; Lu, Yu/AAA-3190-2020","Lu, Yu/0000-0002-5128-6868; Shin, YoungJu/0000-0001-6751-8064",National Institute on Drug Abuse [R01DA005629],National Institute on Drug Abuse(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USANIH National Institute on Drug Abuse (NIDA)),"This manuscript was supported by Grant Numbe

In [10]:
art['text'] = art['Article Title'].fillna('') + ' ' + art['Abstract'].fillna('')
art.head(2)


,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,Book Series Title,Book Series Subtitle,Language,Document Type,Conference Title,Conference Date,Conference Location,Conference Sponsor,Conference Host,Author Keywords,Keywords Plus,Abstract,Addresses,Affiliations,Reprint Addresses,Email Addresses,Researcher Ids,ORCIDs,Funding Orgs,Funding Name Preferred,Funding Text,Cited References,Cited Reference Count,"Times Cited, WoS Core","Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publisher,Publisher City,Publisher Address,ISSN,eISSN,ISBN,Journal Abbreviation,Journal ISO Abbreviation,Publication Date,Publication Year,Volume,Issue,Part Number,Supplement,Special Issue,Meeting Abstract,Start Page,End Page,Article Number,DOI,DOI Link,Book DOI,Early Access Date,Number of Pages,WoS Categories,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record,text
0,J,"Shin, YJ; Lee, JK; Lu, Y; Hecht, ML",NaN,NaN,NaN,"Shin, YoungJu; Lee, Jeong-Kyu; Lu, Yu; Hecht, Michael L.",NaN,NaN,Exploring Parental Influence on the Progression of Alcohol Use in Mexican-Heritage Youth: a Latent Transition Analysis,PREVENTION SCIENCE,NaN,NaN,English,Article,NaN,NaN,NaN,NaN,NaN,Youth substance use; Latent class analysis; Latent transition analysis; Targeted parent-child communication; Parental monitoring,SUBSTANCE USE; DRUG-USE; CHILD COMMUNICATION; FAMILY-STRUCTURE; SAS PROCEDURE; LATINO YOUTH; SELF-ESTEEM; ADOLESCENTS; RISK; MARIJUANA,"Mexican-heritage youth are members of the fastest growing minority group and are at particular risk for substance use including alcohol consumption. Youth face numerous risk factors including positive descriptions of substance use on media and peer offers that are potentially ameliorated by parental anti-substance use socialization efforts. Guided by primary socialization theory and the theory of planned behavior, the present study posited eight research questions to identify discrete subgroups/patterns of Mexican-heritage youth alcohol use behavior and parental influence on youth outcomes. Longitudinal survey data (n = 1147) from youth in 29 public schools located in Phoenix, Arizona, were collected over 3 years. Latent class and transition analyses identified four discrete subgroups characterized by response patterns of alcohol use behaviors and perceptions in Mexican-heritage youth: (1) non-drinker, (2) potential drinker, (3) experimenter, and (4) regular drinker. Targeted parent-child communication about alcohol and parental monitoring were found to be significant predictors for youth alcohol use. Research implications and future directions are suggested.","[Shin, YoungJu] Indiana Univ Purdue Univ, Dept Commun Studies, 425 Univ Blvd,Cavanaugh Hall 309, Indianapolis, IN 46202 USA; [Lee, Jeong-Kyu] Univ Wollongong, Ctr Hlth Initiat, Wollongong, NSW, Australia; [Lu, Yu; Hecht, Michael L.] Penn State Univ, Dept Commun Arts & Sci, University Pk, PA 16802 USA",Indiana University System; Indiana University-Purdue University Indianapolis; University of Wollongong; Pennsylvania Commonwealth System of Higher Education (PCSHE); Pennsylvania State University; Pennsylvania State University - University Park,"Shin, YJ (corresponding author), Indiana Univ Purdue Univ, Dept Commun Studies, 425 Univ Blvd,Cavanaugh Hall 309, Indianapolis, IN 46202 USA.",younshin@iupui.edu; jklee@uow.edu.au; yxl235@psu.edu; mlh10@psu.edu,"Lu, Yu/AGB-5690-2022; Shin, YoungJu/ABA-1000-2020; Lu, Yu/AAA-3190-2020","Lu, Yu/0000-0002-5128-6868; Shin, YoungJu/0000-0001-6751-8064",National Institute on Drug Abuse [R01DA005629],National Institute on Drug Abuse(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USANIH National Institute on Drug Abuse (NIDA)),"This manuscript was supported by Grant 

In [11]:
# Lemmatization
import nltk
import string
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
nltk.download('all')   # download wordlist
from nltk.stem import WordNetLemmatizer
english_stopwords = set(nltk.corpus.stopwords.words('english'))
punctuation = set(string.punctuation)

art['text'] = art['text'].str.replace(r'\b\w{1,3}\b', '') # remove words with less than 3 characters
art['text'] = art['text'].str.replace(r'\d+', '') # remove numbers


lemmatizer = WordNetLemmatizer() 
# tgw = ['alcohol','substance','youth','adolescent','factor','factors','among']  # topic general words to be excluded
tgw = ['alcohol','among']  # topic general words to be excluded

def lemmatizing_en(text):
    word_tokens = word_tokenize(text.lower()) 
    word_tokens = [word for word in word_tokens if word not in punctuation]
    word_tokens = [word for word in word_tokens if word.isalpha()]
    word_tokens = [word for word in word_tokens if word not in english_stopwords]
    word_tokens = [word for word in word_tokens if word not in tgw]
    result = [lemmatizer.lemmatize(x) for x in word_tokens]
    seperator = ' '
    return seperator.join(result)  # return list and string

art['text'] = [lemmatizing_en(s) for s in art['text']]

for word in tgw:
    art['text'] = art['text'].str.replace(word, '') # remove tgw words

art['text'] = art['text'].str.replace(r'\b\w{1,3}\b', '') # remove lemmas with less than 3 characters


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcor864\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\jcor864\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\jcor864\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\jcor864\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\jcor864\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nl

In [12]:
art['text'].iloc[0]

'exploring parental influence progression youth latent transition analysis youth member fastest growing minority group particular risk substance including consumption youth face numerous risk factor including positive description substance medium peer offer potentially ameliorated parental socialization effort guided primary socialization theory theory planned behavior present study posited eight research question identify discrete youth behavior parental influence youth outcome longitudinal survey data youth public school located phoenix arizona collected year latent class transition analysis identified four discrete subgroup characterized response pattern behavior perception youth potential drinker experimenter regular drinker targeted communication parental monitoring found significant predictor youth research implication future direction suggested'

## Bertopic

In [13]:
art.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5596 entries, 0 to 5596
Data columns (total 73 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            5596 non-null   object 
 1   Authors                     5596 non-null   object 
 2   Book Authors                0 non-null      float64
 3   Book Editors                5 non-null      object 
 4   Book Group Authors          0 non-null      float64
 5   Author Full Names           5596 non-null   object 
 6   Book Author Full Names      0 non-null      float64
 7   Group Authors               104 non-null    object 
 8   Article Title               5596 non-null   object 
 9   Source Title                5596 non-null   object 
 10  Book Series Title           5 non-null      object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    5596 non-null   object 
 13  Document Type               5596 

In [14]:

from bertopic import BERTopic

# Pre-calculate embeddings
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# embeddings = embedding_model.encode(art['text'], show_progress_bar=True)

# dimensionality reduction
from umap import UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)    # Prevent Stochastic Behavior with random_state

# clustering (controlling number of topics)
# from hdbscan import HDBSCAN
# hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Improve topic representations
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2))

# Fine-tune your topic representations
from bertopic.representation import KeyBERTInspired
representation_model = KeyBERTInspired()

topic_model = BERTopic(

  # Pipeline models
  # embedding_model=embedding_model,
  umap_model=umap_model,
  # hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  nr_topics=nr_topics,
  top_n_words=10,
  verbose=True
)

topics, probs = topic_model.fit_transform(art['text'])



2024-05-03 12:08:02,377 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/175 [00:00<?, ?it/s]

2024-05-03 12:11:02,207 - BERTopic - Embedding - Completed ✓
2024-05-03 12:11:02,208 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-03 12:11:25,327 - BERTopic - Dimensionality - Completed ✓
2024-05-03 12:11:25,327 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-03 12:11:25,501 - BERTopic - Cluster - Completed ✓
2024-05-03 12:11:25,502 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-03 12:11:49,024 - BERTopic - Representation - Completed ✓
2024-05-03 12:11:49,028 - BERTopic - Topic reduction - Reducing number of topics
2024-05-03 12:11:55,693 - BERTopic - Topic reduction - Reduced number of topics from 84 to 11


In [15]:
topic_model.save(path=".", serialization="safetensors", save_ctfidf=True)

In [16]:
nr_topics

11

In [17]:
probs[:20]

array([0.        , 0.45513667, 0.        , 0.        , 0.        ,
       1.        , 1.        , 0.        , 0.72604774, 0.91888176,
       0.8847519 , 0.98689247, 0.10145557, 0.        , 0.        ,
       1.        , 0.41657135, 0.        , 0.        , 0.        ])

In [18]:
opt_nr_topics = len(set(topics))
opt_nr_topics

11

In [19]:
# topic information
for i in range(0, opt_nr_topics-1):
    print(f"Topic {i}")
    for j in range(0,10):
        word = topic_model.get_topic(i)[j][0]
        print(f'{word.ljust(20, " ")} \t {round(topic_model.get_topic(i)[j][1],4)}')
    print('')


Topic 0
adolescent           	 0.47859999537467957
adolescence          	 0.4311999976634979
risk factor          	 0.35019999742507935
youth                	 0.3422999978065491
parental             	 0.3393000066280365
prevalence           	 0.30070000886917114
adulthood            	 0.29670000076293945
prevention           	 0.28600001335144043
health               	 0.2732999920845032
parent               	 0.2581000030040741

Topic 1
prenatal exposure    	 0.4641000032424927
adhd                 	 0.4244000017642975
disorder adhd        	 0.41100001335144043
cocaine exposure     	 0.3580999970436096
prenatal             	 0.3425999879837036
developmental        	 0.2904999852180481
marijuana            	 0.26010000705718994
maternal             	 0.25
adolescent           	 0.24570000171661377
pregnancy            	 0.24120000004768372

Topic 2
fatty liver          	 0.42829999327659607
insulin resistance   	 0.36070001125335693
liver disease        	 0.352400004863739
obesity     

In [20]:
topic_model.visualize_barchart(top_n_topics=10, n_words=10,width=500, height=500)

In [21]:
try:
    print(topic_model.visualize_topics())
except:
    print('Topics kunnen niet gevisualiseerd worden.')
    pass

Figure({
    'data': [{'customdata': array([[0, 'adolescent | adolescence | risk factor | youth | parental', 3118],
                                   [1,
                                    'prenatal exposure | adhd | disorder adhd | cocaine exposure | prenatal',
                                    153],
                                   [2,
                                    'fatty liver | insulin resistance | liver disease | obesity | obese child',
                                    152],
                                   [3, 'fermentation | ethanol | wine | grape | yeast', 116],
                                   [4,
                                    'ethanol exposure | intermittent ethanol | ethanol | hippocampal | hippocampus',
                                    100],
                                   [5,
                                    'liver disease | fatty liver | hepatic steatosis | lipid profile | hepatic',
                                    75],
               

In [22]:
topic_model.visualize_topics()

In [23]:
tm = topic_model.get_topic_info()[['Topic', 'Count', 'Name','Representation']]
tm

,Topic,Count,Name,Representation
0,-1,1728,-1_adolescent_adolescence_youth_risk factor,"[adolescent, adolescence, youth, risk factor, adulthood, parental, prevalence, prevention, intervention, assessed]"
1,0,3118,0_adolescent_adolescence_risk factor_youth,"[adolescent, adolescence, risk factor, youth, parental, prevalence, adulthood, prevention, health, parent]"
2,1,153,1_prenatal exposure_adhd_disorder adhd_cocaine exposure,"[prenatal exposure, adhd, disorder adhd, cocaine exposure, prenatal, developmental, marijuana, maternal, adolescent, pregnancy]"
3,2,152,2_fatty liver_insulin resistance_liver disease_obesity,"[fatty liver, insulin resistance, liver disease, obesity, obese child, obese, cardiovascular risk, insulin, diabetes, metabolic]"
4,3,116,3_fermentation_ethanol_wine_grape,"[fermentation, ethanol, wine, grape, yeast, phenolic, aroma, acetate, biosynthesis, extraction]"
5,4,100,4_ethanol exposure_intermittent ethanol_ethanol_hippocampal,"[ethanol exposure, intermittent ethanol, ethanol, hippocampal, hippocampus, microglial, neurogenesis, microglia, adolescent binge, adolescent exposure]"
6,5,75,5_liver disease_fatty liver_hepatic steatosis_lipid profile,"[liver disease, fatty liver, hepatic steatosis, lipid profile, hepatic, genotype, liver, cholesterol, nucleotide polymorphism, cirrhosis]"
7,6,61,6_adolescent sleep_sleep disturbance_association sleep_sleep problem,"[adolescent sleep, sleep disturbance, association sleep, sleep problem, sleep difficulty, sleep quality, sleep loss, sleepiness, daytime sleepiness, sleep duration]"
8,7,41,7_injury severity_trauma case_trauma_hospitalization,"[injury severity, trauma case, trauma, hospitalization, severity, emergency department, injury, fracture, accident, hospital]"
9,8,41,8_risk breast_cancer risk_breast cancer_smoking consumption,"[risk breast, cancer risk, breast cancer, smoking consumption, risk factor, increased risk, cancer, risk, cancer case, colorectal cancer]"


In [24]:
tm.to_excel('../topics.xlsx', index=False)

In [25]:
# Run the visualization with the original embeddings
# topic_model.visualize_documents(art['text'], embeddings=embeddings)

In [26]:
topic_model.visualize_heatmap()

In [27]:
probs[:20]

array([0.        , 0.45513667, 0.        , 0.        , 0.        ,
       1.        , 1.        , 0.        , 0.72604774, 0.91888176,
       0.8847519 , 0.98689247, 0.10145557, 0.        , 0.        ,
       1.        , 0.41657135, 0.        , 0.        , 0.        ])

## Attaching Discovered Topic Labels to Original Articles

In [28]:
art['Topic'] = topics
art['Prob'] = probs
# art = art[(art['Topic'] >= 0)]   # -1 refers to all outliers and should typically be ignored. 
art = art.drop(columns=['text'])

In [29]:
art[art['Topic'] == 1][['Article Title','Abstract','Prob']][:10]

,Article Title,Abstract,Prob
130,Emotional Processing and Brain Activity in Youth at High Risk for Alcoholism,"Background: Even in the absence of heavy alcohol use, youth with familial alcoholism (family history positive [FHP]) exhibit atypical brain functioning and behavior. Although emotional and cognitive systems are affected in alcohol use disorders (AUDs), little attention has focused on whether brain and behavior phenotypes related to the interplay between affective and executive functioning may be a premorbid risk factor for the development of AUDs in FHP youth. Methods: Twenty-four FHP and 22 family history negative (FHN) 12- to 16-year-old adolescents completed study procedures. After exclusion of participants with clinically significant depressive symptoms and those who did not meet performance criteria during an Emotional Go-NoGo task, 19 FHP and 17 FHN youth were included in functional magnetic resonance imaging (fMRI) analyses. Resting state functional connectivity MRI, using amygdalar seed regions, was analyzed in 16 FHP and 18 FHN youth, after exclusion of participants with excessive head movement. Results: fMRI showed that brain activity in FHP youth, compared with FHN peers, was reduced during emotional processing in the superior temporal cortex, as well as during cognitive control within emotional contexts in frontal and striatal regions. Group differences in resting state amygdalar connectivity were seen bilaterally between FHP and FHN youth. In FHP youth, reduced resting state synchrony between the left amygdala and left superior frontal gyrus was related to poorer response inhibition, as measured during the fMRI task. Conclusions: To our knowledge, this is the first study to examine emotion-cognition interactions and resting state functional connectivity in FHP youth. Findings from this research provide insight into neural and behavioral phenotypes associated with emotional processing in familial alcoholism, which may relate to increased risk of developing AUDs.",0.584518
141,"Prenatal substance exposure, adverse childhood experiences and diagnosed mental and behavioral disorders - A longitudinal register-based matched cohort study in Finland","Both prenatal substance exposure (PSE, alcohol/drugs) and experiences during the first years of life have powerful effects on brain development. However, only a few studies have investigated the combined effect of PSE and adverse childhood experiences (ACEs) on mental and behavioral disorders among exposed adolescents and adults. This longitudinal register-based cohort study 1) compared the nature and extent of diagnosed mental and behavioral disorders among youth with PSE and matched unexposed controls, and 2) investigated the influence of PSE, health in infancy and ACEs (maternal risk factors and out-of-home care, OHC) on diagnoses of mental and behavioral disorders. The data consisted of 615 exposed youth aged 15-24 years and 1787 matched unexposed controls. Data from hospital medical records and nine registers were merged for the analysis. Descriptive analysis methods and Cox regression were used. The results showed that the prevalence of mental and behavioral disorders was twice as high among exposed compared with controls. The highest levels of mental and behavioral disorders and ACEs were found among exposed with at least one OHC episode. The difference in the risk of mental and behavioral disorders between exposed and controls diminished after controlling for the effect of ACEs. Low birth weight, maternal risk factors, and OHC were the strongest predictors of mental and behavioral disorders. The results suggest that PSE alone does not explain poorer mental health among exposed youth. Risk factors accumulate, and low birth weight and ACEs are strongly associated with increased risk of mental and behavioral disorders.",0.688872
147,The Relationship Between Socioeconomic Status and Brain Volume in Children and Adolescents With Prenatal Alcohol Exposure,"The pos

## Number of articles per topic and per year

In [30]:
# pivot table year x topic --> count

pvt = pd.pivot_table(art, columns="Topic", index="Publication Year", values='Abstract',aggfunc='count', margins=True).fillna(0).astype(int)
pvt

Topic,-1,0,1,2,3,4,5,6,7,8,9,All
Publication Year,,,,,,,,,,,,
2000,11,10,1,2,0,0,0,0,0,0,0,24
2001,14,10,3,0,2,0,1,1,0,0,0,31
2002,12,16,0,2,3,0,1,2,0,0,0,36
2003,9,16,0,0,1,0,1,0,1,0,0,28
2004,8,13,1,0,0,0,0,2,0,0,0,24
2005,13,17,1,2,1,0,0,1,0,1,0,36
2006,25,35,2,1,1,0,2,0,1,0,0,67
2007,28,55,4,0,1,1,3,0,0,1,0,93
2008,47,82,4,1,1,2,4,0,1,1,0,143


In [31]:
pvt.to_excel('../pvt.xlsx')

### Time trends

In [32]:
pvt1 = pvt.reset_index()
pvt2 = pvt1.copy()

pvt1 = pvt1.drop('All',axis=1)
# pvt1 = pvt1[(pvt1['Publication Year'] != 2024)]
pvt1 = pvt1[:-1]
pvt1

Topic,Publication Year,-1,0,1,2,3,4,5,6,7,8,9
0,2000,11,10,1,2,0,0,0,0,0,0,0
1,2001,14,10,3,0,2,0,1,1,0,0,0
2,2002,12,16,0,2,3,0,1,2,0,0,0
3,2003,9,16,0,0,1,0,1,0,1,0,0
4,2004,8,13,1,0,0,0,0,2,0,0,0
5,2005,13,17,1,2,1,0,0,1,0,1,0
6,2006,25,35,2,1,1,0,2,0,1,0,0
7,2007,28,55,4,0,1,1,3,0,0,1,0
8,2008,47,82,4,1,1,2,4,0,1,1,0
9,2009,41,92,2,2,2,0,2,1,0,1,0


In [33]:
pvt1.to_csv('../pvt_' + str(zoekstring) + '.csv', index=False)

NameError: name 'zoekstring' is not defined

In [ ]:
pvt1.shape

In [ ]:
from matplotlib import pyplot as plt
# set figure size as 8 x 20 inches
plt.figure(figsize=(20,8))
bottom = np.zeros(pvt1.shape[0])
for i in range(0, opt_nr_topics-1):
    plt.bar(pvt1['Publication Year'], pvt1[i], bottom = bottom, label=f'Topic {i}')
    bottom += pvt1[i]

plt.xlabel('Year')
plt.ylabel('Number of Articles')
plt.title('Number of Articles per Topic per Year')
# set ticks on x as years
plt.xticks(np.arange(pvt1['Publication Year'].min(), pvt1['Publication Year'].max()+1, 2))
plt.legend()
plt.grid()
plt.show()

In [ ]:
pvt2['All'] = pvt2['All'] - pvt2[-1]

for i in range(0, opt_nr_topics-1):
    pvt2[i] = pvt2[i] / pvt2['All']

pvt2 = pvt2.drop('All',axis=1)
# pvt2 = pvt2[(pvt2['Publication Year'] != 2024)]
pvt2 = pvt2[:-1]
pvt2



In [ ]:
from matplotlib import pyplot as plt
# set figure size as 8 x 20 inches
plt.figure(figsize=(20,8))

bottom = np.zeros(pvt1.shape[0])
for i in range(0, opt_nr_topics-1):
    plt.bar(pvt2['Publication Year'], pvt2[i], bottom=bottom, label=f'Topic {i}')
    bottom += pvt2[i]

plt.xlabel('Year')
plt.ylabel('Relative number of Articles')
plt.title('Relative number of Articles per Topic per Year')
# set ticks on x as years
plt.xticks(np.arange(pvt2['Publication Year'].min(), pvt2['Publication Year'].max()+1, 2))
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.grid()
plt.show()

In [ ]:
art.to_excel('.' + '.xlsx',index=False)

#### References

- LDA gensim vs scikit-learn: https://blog.devgenius.io/working-with-sklearn-latent-dirichlet-allocation-lda-topic-modeling-on-airline-sentiment-twitter-56dfacfd58bf  - 
- Perplexity score: https://medium.com/@iqra.bismi/topic-modelling-using-lda-fe81a2a806e0#:~:text=Perplexity%20is%20a%20commonly%20used,score%20indicates%20better%20model%20performance
- A Topic Modeling Comparison Between LDA, NMF, Top2Vec, and BERTopic to Demystify Twitter Posts: https://www.frontiersin.org/articles/10.3389/fsoc.2022.886498/full
- https://towardsdatascience.com/visualizing-topic-models-with-scatterpies-and-t-sne-f21f228f7b02
- https://maartengr.github.io/BERTopic/index.html 